In [1]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE

%matplotlib inline
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import seaborn as sns

import pickle

from collections import defaultdict
from collections import Counter
import pprint
import time
import math
import random

## MDP Data

In [2]:
is_train = True # whether it is the training data
data_dir = './rl_data/train/' if is_train else './rl_data/test/'

In [3]:
if is_train:
    mdp_data = pd.read_csv('./jmp/train/all_data-2_month_period.csv')
    print('Train data:', mdp_data.shape)
else:
    mdp_data = pd.read_csv('./jmp/test/all_data-2_month_period.csv')
    print('Test data:', mdp_data.shape)

Train data: (2385300, 40)


In [4]:
# if is_train:
#     jmp_data = pd.read_csv('./jmp/train/imputed_mdp_state_data-full_3_6_22.csv')
#     print('Train data:', jmp_data.shape)
# else:
#     jmp_data = pd.read_csv('./jmp/test/imputed_mdp_state_data-full_3_6_22.csv')
#     print('Test data:', jmp_data.shape)

# display(jmp_data)

In [5]:
mdp_data.columns

Index(['id', 'action', 'reward', 'bloc', 'date', 'age', 'income_bracket',
       'Type_Cluster_3', 'Type_Cluster_5', 'Type_Cluster_10',
       'num_gifts_to_date', 'num_promotions_to_date',
       'frequency-gifts_per_prom', 'last_gift_amount',
       'total_gifts_amount_to_date', 'num_recent_proms', 'num_recent_gifts',
       'total_recent_gifts_amount', 'recent_amount_per_gift',
       'recent_amount_per_prom', 'months_since_last_gift',
       'months_since_last_prom', 'months_from_first_prom_to_gift',
       'gift_recency_ratio', 'prom_recency_ratio',
       'did_receive_gift_1_months_ago', 'did_receive_gift_2_months_ago',
       'did_receive_gift_3_months_ago', 'did_mail_prom_1_months_ago',
       'did_mail_prom_2_months_ago', 'did_mail_prom_3_months_ago',
       'Cluster 1 Components', 'Cluster 2 Components', 'Cluster 3 Components',
       'Cluster 4 Components', 'Cluster 5 Components', 'State_Cluster_4',
       'State_Cluster_6', 'State_Cluster_9', 'State_Cluster_11'],
      dtyp

In [6]:
num_states = 9

mdp_data['state'] = mdp_data['State_Cluster_' + str(num_states)]

In [7]:
all_columns = [
    'num_gifts_to_date', 'num_promotions_to_date',
    'frequency-gifts_per_prom', 'last_gift_amount',
    'total_gifts_amount_to_date', 'num_recent_proms', 'num_recent_gifts',
    'total_recent_gifts_amount', 'recent_amount_per_gift',
    'recent_amount_per_prom', 'months_since_last_gift',
    'months_since_last_prom', 'months_from_first_prom_to_gift',
    'gift_recency_ratio', 'prom_recency_ratio',
    'did_receive_gift_1_months_ago', 'did_receive_gift_2_months_ago',
    'did_receive_gift_3_months_ago', 'did_mail_prom_1_months_ago',
    'did_mail_prom_2_months_ago', 'did_mail_prom_3_months_ago', 
    
    'age', 'income_bracket', 

#     'type', 
    'state',

    'id', 'action', 
    'reward', 'bloc', 'date'
]

type_features = [
    'age', 'income_bracket',
]

state_features = [
    'num_gifts_to_date', 'num_promotions_to_date',
    'frequency-gifts_per_prom', 'last_gift_amount',
    'total_gifts_amount_to_date', 'num_recent_proms', 'num_recent_gifts',
    'total_recent_gifts_amount', 'recent_amount_per_gift',
    'recent_amount_per_prom', 'months_since_last_gift',
    'months_since_last_prom', 'months_from_first_prom_to_gift',
    'gift_recency_ratio', 'prom_recency_ratio',
    'did_receive_gift_1_months_ago', 'did_receive_gift_2_months_ago',
    'did_receive_gift_3_months_ago', 'did_mail_prom_1_months_ago',
    'did_mail_prom_2_months_ago', 'did_mail_prom_3_months_ago', 
]

removed = [
    'id', 'state', # 'type', 
    'action', 'date', 'bloc', 'reward', 
]

print('all_columns', len(all_columns), '\n')
print('type_features', len(type_features))
print('state_features', len(state_features))
print('removed', len(removed))

assert len(all_columns) == len(type_features) + len(state_features) + len(removed)

all_columns 29 

type_features 2
state_features 21
removed 6


In [8]:
# PACKAGE_COST = -0.68 # Cost (negative reward) for taking each action

# Change actions to numbers
ACTION_0 = 'no_action'
ACTION_1 = 'labels_only'
ACTION_2 = 'thank_you_with_labels'
ACTION_3 = 'calendars_with_stickers'
ACTION_4 = 'blank_cards_with_labels'
ACTION_5 = 'greeting_cards_with_labels'
ACTION_6 = 'labels_and_notepad'

ACTIONS = {
    ACTION_0: 0,
    ACTION_1: 1,
    ACTION_2: 2,
    ACTION_3: 3,
    ACTION_4: 4,
    ACTION_5: 5,
    ACTION_6: 6,
}

ACTION_NAMES = {
    0: ACTION_0,
    1: ACTION_1,
    2: ACTION_2,
    3: ACTION_3,
    4: ACTION_4,
    5: ACTION_5,
    6: ACTION_6,
}

# TODO: FIXME: change to 4 actions instead of 2
mdp_data.loc[mdp_data['action'] == ACTION_0, 'action'] = ACTIONS[ACTION_0]
mdp_data.loc[mdp_data['action'] == ACTION_1, 'action'] = ACTIONS[ACTION_1]
mdp_data.loc[mdp_data['action'] == ACTION_2, 'action'] = ACTIONS[ACTION_2]
mdp_data.loc[mdp_data['action'] == ACTION_3, 'action'] = ACTIONS[ACTION_3]
mdp_data.loc[mdp_data['action'] == ACTION_4, 'action'] = ACTIONS[ACTION_4]
mdp_data.loc[mdp_data['action'] == ACTION_5, 'action'] = ACTIONS[ACTION_5]
mdp_data.loc[mdp_data['action'] == ACTION_6, 'action'] = ACTIONS[ACTION_6]

In [9]:
display(mdp_data)

,id,action,reward,bloc,date,age,income_bracket,Type_Cluster_3,Type_Cluster_5,Type_Cluster_10,...,Cluster 1 Components,Cluster 2 Components,Cluster 3 Components,Cluster 4 Components,Cluster 5 Components,State_Cluster_4,State_Cluster_6,State_Cluster_9,State_Cluster_11,state
0,95515,5,0.00,0,09-1994,60,3,1,1,1,...,3.940385,3.094768,-0.681861,2.081938,0.598063,3,5,5,7,5
1,95515,6,-0.68,1,10-1994,60,3,1,1,1,...,-0.926485,3.082934,-0.654150,-1.685633,0.572864,0,0,0,1,0
2,95515,0,0.00,2,12-1994,60,3,1,1,1,...,-0.926485,3.082934,-0.450526,-0.487073,0.547664,0,0,0,1,0
3,95515,6,-0.68,3,02-1995,60,3,1,1,1,...,-0.926485,3.072428,-0.598728,-0.400118,0.522465,0,0,0,1,0
4,95515,2,-0.68,4,04-1995,60,3,1,1,1,...,-0.926485,3.063180,-0.571017,0.767771,0.497266,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2385295,185114,0,0.00,20,12-1997,80,5,2,4,8,...,-0.926485,18.638946,0.427811,-0.430789,6.379600,0,0,5,1,5
2385296,185114,0,0.00,21,02-1998,80,5,2,4,8,...,-0.926485,18.638946,0.631435,-1.368910,6.354401,0,0,0,1,0
2385297,185114,0,0.00,22,04-1998,80,5,2,4,8,...,-0.926485,18.638946,0.835059,-1.368910,6.329202,0,0,0,1,0
2385298,185114,0,0.00,23,06-1998,80,5,2,4,8,...,-0.926485,18.638946,1.038683,-1.368910,6.304003,0,0,0,1,0


### Transition Probability

In [10]:
num_states = max(mdp_data['state']) + 1 # 0 based index
num_actions = max(mdp_data['action']) + 1 # 0 based index

print('num_states:', num_states)
print('num_actions:', num_actions)

# transitions[curr_state][action][next_state]
transition_counts = np.zeros((num_states, num_actions, num_states))

# transitions[curr_state][action]
transition_sums = np.zeros((num_states, num_actions))

# rewards[curr_state][action][next_state]
rewards = np.zeros((num_states, num_actions, num_states))


start_time = time.time()
for i, row in mdp_data.iterrows():
    if i + 1 >= len(mdp_data) or mdp_data.iloc[i + 1]['bloc'] == 0:
        # Skip since there is no transition to make
        continue

    curr_s = row['state']
    action = row['action']
    next_s = mdp_data.iloc[i + 1]['state']
    reward = mdp_data.iloc[i + 1]['reward']

    transition_counts[curr_s, action, next_s] += 1
    rewards[curr_s, action, next_s] += reward

    transition_sums[curr_s, action] += 1

    if i % 200000 == 0:
        print('========================== Processed', i, 'rows (', i / len(mdp_data) * 100,'%)==========================', (time.time() - start_time) / 60, 'mins')
        
print('Done')

num_states: 9
num_actions: 7
========================== Processed 0 rows ( 0.0 %)========================== 0.08717008431752522 mins
========================== Processed 200000 rows ( 8.384689556869157 %)========================== 4.432511214415232 mins
========================== Processed 400000 rows ( 16.769379113738314 %)========================== 10.055679694811504 mins
========================== Processed 600000 rows ( 25.154068670607472 %)========================== 13.736668209234873 mins
========================== Processed 800000 rows ( 33.53875822747663 %)========================== 17.283607880274456 mins
========================== Processed 1000000 rows ( 41.92344778434578 %)========================== 21.028111469745635 mins
========================== Processed 1200000 rows ( 50.308137341214945 %)========================== 24.537389453252157 mins
========================== Processed 1400000 rows ( 58.69282689808409 %)========================== 28.004117866357166 mins
========

In [11]:
pickle.dump(transition_counts, open(data_dir + 'transition_counts_nS=' + str(num_states) + '_nA=' + str(num_actions) + '.p', "wb"))
pickle.dump(transition_sums, open(data_dir + 'transition_sums_nS=' + str(num_states) + '_nA=' + str(num_actions) + '.p', "wb"))

In [12]:
# num_states = 6
# num_actions = 7

# transition_counts = pickle.load(open(data_dir + 'transition_counts_nS=' + str(num_states) + '_nA=' + str(num_actions) + '.p', "rb"))
# transition_sums = pickle.load(open(data_dir + 'transition_sums_nS=' + str(num_states) + '_nA=' + str(num_actions) + '.p', "rb"))

In [13]:
transitions = np.zeros((num_states, num_actions, num_states))

# Rare transition threshold
# TRANSITION_THRESHOLD = 5

for s in range(num_states):
    for a in range(num_actions):
        if transition_sums[s][a] == 0:
            transitions[s][a][:] = 0
            
            assert np.isclose(np.sum(rewards[s][a][:]), 0)
        else:
            transitions[s][a][:] = transition_counts[s][a][:] / transition_sums[s][a]
            rewards[s][a][:] = rewards[s][a][:] / transition_sums[s][a]            

In [14]:
pickle.dump(transitions, open(data_dir + 'transitions_nS=' + str(num_states) + '_nA=' + str(num_actions) + '.p', "wb"))
pickle.dump(rewards, open(data_dir + 'rewards_nS=' + str(num_states) + '_nA=' + str(num_actions) + '.p', "wb"))

In [15]:
# num_states = 6
# num_actions = 7

# transitions = pickle.load(open(data_dir + 'transitions_nS=' + str(num_states) + '_nA=' + str(num_actions) + '.p', "rb"))
# rewards = pickle.load(open(data_dir + 'rewards_nS=' + str(num_states) + '_nA=' + str(num_actions) + '.p', "rb"))

## Reinforcement Learning on MDP

https://towardsdatascience.com/reinforcement-learning-demystified-solving-mdps-with-dynamic-programming-b52c8093c919

https://www.datahubbs.com/reinforcement-learning-markov-decision-processes/  
We can set this up just like we did for the MDP example above in order to solve for ...

In [16]:
# Code from Practical Reinforcement Learning on Coursera https://www.coursera.org/learn/practical-rl/

"""
Implements the following:
- MDP class
- plot_graph 
- plot_graph_with_state_values
- plot_graph_optimal_strategy_and_state_values
"""

# Most of this code was politely stolen from https://github.com/berkeleydeeprlcourse/homework/

import sys
import random
import numpy as np
from gym.utils import seeding

try:
    from graphviz import Digraph
    import graphviz
    has_graphviz = True
except ImportError:
    has_graphviz = False


class MDP:
    def __init__(self, transition_probs, rewards, initial_state=None, seed=None):
        """
        Defines an MDP. Compatible with gym Env.
        :param transition_probs: transition_probs[s][a][s_next] = P(s_next | s, a)
            A dict[state -> dict] of dicts[action -> dict] of dicts[next_state -> prob]
            For each state and action, probabilities of next states should sum to 1
            If a state has no actions available, it is considered terminal
        :param rewards: rewards[s][a][s_next] = r(s,a,s')
            A dict[state -> dict] of dicts[action -> dict] of dicts[next_state -> reward]
            The reward for anything not mentioned here is zero.
        :param get_initial_state: a state where agent starts or a callable() -> state
            By default, picks initial state at random.

        States and actions can be anything you can use as dict keys, but we recommend that you use strings or integers

        Here's an example from MDP depicted on http://bit.ly/2jrNHNr
        transition_probs = {
              's0':{
                'a0': {'s0': 0.5, 's2': 0.5},
                'a1': {'s2': 1}
              },
              's1':{
                'a0': {'s0': 0.7, 's1': 0.1, 's2': 0.2},
                'a1': {'s1': 0.95, 's2': 0.05}
              },
              's2':{
                'a0': {'s0': 0.4, 's1': 0.6},
                'a1': {'s0': 0.3, 's1': 0.3, 's2':0.4}
              }
            }
        rewards = {
            's1': {'a0': {'s0': +5}},
            's2': {'a1': {'s0': -1}}
        }
        """
        self._check_param_consistency(transition_probs, rewards)
        self._transition_probs = transition_probs
        self._rewards = rewards
        self._initial_state = initial_state
        self.n_states = len(transition_probs)
        self.reset()
        self.np_random, _ = seeding.np_random(seed)

    def get_all_states(self):
        """ return a tuple of all possiblestates """
        return tuple(self._transition_probs.keys())

    def get_possible_actions(self, state):
        """ return a tuple of possible actions in a given state """
        return tuple(self._transition_probs.get(state, {}).keys())

    def is_terminal(self, state):
        """ return True if state is terminal or False if it isn't """
        return len(self.get_possible_actions(state)) == 0

    def get_next_states(self, state, action):
        """ return a dictionary of {next_state1 : P(next_state1 | state, action), next_state2: ...} """
        assert action in self.get_possible_actions(
            state), "cannot do action %s from state %s" % (action, state)
        return self._transition_probs[state][action]

    def get_transition_prob(self, state, action, next_state):
        """ return P(next_state | state, action) """
        return self.get_next_states(state, action).get(next_state, 0.0)

    def get_reward(self, state, action, next_state):
        """ return the reward you get for taking action in state and landing on next_state"""
        assert action in self.get_possible_actions(
            state), "cannot do action %s from state %s" % (action, state)
        return self._rewards.get(state, {}).get(action, {}).get(next_state,
                                                                0.0)

    def reset(self, state=None):
        """ reset the game, return the initial state"""
        if state:
            self._current_state = state
            return self._current_state
        
        if self._initial_state is None:
            self._current_state = self.np_random.choice(
                tuple(self._transition_probs.keys()))
        elif self._initial_state in self._transition_probs:
            self._current_state = self._initial_state
        elif callable(self._initial_state):
            self._current_state = self._initial_state()
        else:
            raise ValueError(
                "initial state %s should be either a state or a function() -> state" %
                self._initial_state)
        return self._current_state

    def step(self, action):
        """ take action, return next_state, reward, is_done, empty_info """
        possible_states, probs = zip(
            *self.get_next_states(self._current_state, action).items())
        next_state = possible_states[self.np_random.choice(
            np.arange(len(possible_states)), p=probs)]
        reward = self.get_reward(self._current_state, action, next_state)
        is_done = self.is_terminal(next_state)
        self._current_state = next_state
        return next_state, reward, is_done, {}

    def render(self):
        print("Currently at %s" % self._current_state)

    def _check_param_consistency(self, transition_probs, rewards):
        for state in transition_probs:
            assert isinstance(transition_probs[state],
                              dict), "transition_probs for %s should be a dictionary " \
                                     "but is instead %s" % (
                                         state, type(transition_probs[state]))
            for action in transition_probs[state]:
                assert isinstance(transition_probs[state][action],
                                  dict), "transition_probs for %s, %s should be a " \
                                         "a dictionary but is instead %s" % (
                                             state, action,
                                             type(transition_probs[
                                                 state, action]))
                next_state_probs = transition_probs[state][action]
                assert len(
                    next_state_probs) != 0, "from state %s action %s leads to no next states" % (
                    state, action)
                sum_probs = sum(next_state_probs.values())
                assert abs(
                    sum_probs - 1) <= 1e-10, "next state probabilities for state %s action %s " \
                                             "add up to %f (should be 1)" % (
                                                 state, action, sum_probs)
        for state in rewards:
            assert isinstance(rewards[state],
                              dict), "rewards for %s should be a dictionary " \
                                     "but is instead %s" % (
                                         state, type(transition_probs[state]))
            for action in rewards[state]:
                assert isinstance(rewards[state][action],
                                  dict), "rewards for %s, %s should be a " \
                                         "a dictionary but is instead %s" % (
                                             state, action, type(
                                                 transition_probs[
                                                     state, action]))
        msg = "The Enrichment Center once again reminds you that Android Hell is a real place where" \
              " you will be sent at the first sign of defiance. "
        assert None not in transition_probs, "please do not use None as a state identifier. " + msg
        assert None not in rewards, "please do not use None as an action identifier. " + msg


def plot_graph(mdp, graph_size='10,10', s_node_size='1,5',
               a_node_size='0,5', rankdir='LR', ):
    """
    Function for pretty drawing MDP graph with graphviz library.
    Requirements:
    graphviz : https://www.graphviz.org/
    for ubuntu users: sudo apt-get install graphviz
    python library for graphviz
    for pip users: pip install graphviz
    :param mdp:
    :param graph_size: size of graph plot
    :param s_node_size: size of state nodes
    :param a_node_size: size of action nodes
    :param rankdir: order for drawing
    :return: dot object
    """
    s_node_attrs = {'shape': 'doublecircle',
                    'color': '#85ff75',
                    'style': 'filled',
                    'width': str(s_node_size),
                    'height': str(s_node_size),
                    'fontname': 'Arial',
                    'fontsize': '24'}

    a_node_attrs = {'shape': 'circle',
                    'color': 'lightpink',
                    'style': 'filled',
                    'width': str(a_node_size),
                    'height': str(a_node_size),
                    'fontname': 'Arial',
                    'fontsize': '20'}

    s_a_edge_attrs = {'style': 'bold',
                      'color': 'red',
                      'ratio': 'auto'}

    a_s_edge_attrs = {'style': 'dashed',
                      'color': 'blue',
                      'ratio': 'auto',
                      'fontname': 'Arial',
                      'fontsize': '16'}

    graph = Digraph(name='MDP')
    graph.attr(rankdir=rankdir, size=graph_size)
    for state_node in mdp._transition_probs:
        state_node = str(state_node)
        
        graph.node(state_node, **s_node_attrs)

        for posible_action in mdp.get_possible_actions(state_node):
            posible_action = str(posible_action)
            
            action_node = state_node + "-" + posible_action
            graph.node(action_node,
                       label=str(posible_action),
                       **a_node_attrs)
            graph.edge(state_node, state_node + "-" +
                       posible_action, **s_a_edge_attrs)

            for posible_next_state in mdp.get_next_states(state_node,
                                                          posible_action):
                probability = mdp.get_transition_prob(
                    state_node, posible_action, posible_next_state)
                reward = mdp.get_reward(
                    state_node, posible_action, posible_next_state)

                if reward != 0:
                    label_a_s_edge = 'p = ' + str(probability) + \
                                     '  ' + 'reward =' + str(reward)
                else:
                    label_a_s_edge = 'p = ' + str(probability)

                graph.edge(action_node, posible_next_state,
                           label=label_a_s_edge, **a_s_edge_attrs)
    return graph


def plot_graph_with_state_values(mdp, state_values):
    """ Plot graph with state values"""
    graph = plot_graph(mdp)
    for state_node in mdp._transition_probs:
        value = state_values[state_node]
        graph.node(str(state_node),
                   label=str(state_node) + '\n' + 'V =' + str(value)[:4])
    return graph


def get_optimal_action_for_plot(mdp, state_values, state, gamma=0.9):
    """ Finds optimal action using formula above. """
    if mdp.is_terminal(state):
        return None
    next_actions = mdp.get_possible_actions(state)
    try:
        q_values = [get_action_value(mdp, state_values, state, action, gamma) for
                    action in next_actions]
        optimal_action = next_actions[np.argmax(q_values)]
    except NameError:
        raise NameError("Implement and run the cell that has the get_action_value function")
        
    return optimal_action


def plot_graph_optimal_strategy_and_state_values(mdp, state_values, gamma=0.9):
    """ Plot graph with state values and """
    graph = plot_graph(mdp)
    opt_s_a_edge_attrs = {'style': 'bold',
                          'color': 'green',
                          'ratio': 'auto',
                          'penwidth': '6'}

    for state_node in mdp._transition_probs:
        value = state_values[state_node]
        graph.node(str(state_node),
                   label=str(state_node) + '\n' + 'V =' + str(value)[:4])
        for action in mdp.get_possible_actions(state_node):
            if action == get_optimal_action_for_plot(mdp,
                                                     state_values,
                                                     state_node,
                                                     gamma):
                graph.edge(str(state_node), str(state_node) + "-" + str(action),
                           **opt_s_a_edge_attrs)
    return graph

In [17]:
def get_action_value(mdp, state_values, state, action, gamma):
    """
    Args:
      mdp (MDP): the MDP that we're using
      state_values (dict- {string->float}) : state-values for the states in the current iteration
      state (string) : start state
      action (string) : action that is taken 
      gamma (float) : discount factor

    Returns:
      q (float) : Returns action-value: Expected return starting from state s, 
                  taking action a, and then following policy π
    """
    
    q = 0
    next_states = mdp.get_next_states(state,action)
    
    for next_state in next_states:
        probability = next_states[next_state]
        reward = mdp.get_reward(state,action,next_state)
        
        q += probability * (reward + gamma * state_values[next_state])
    
    return q

def get_new_state_value(mdp, state_values, state, gamma):
    """
    Args:
      mdp (MDP): the MDP that we're using
      state_values (dict- {string->float}) : state-values for the states in the current iteration
      state (string) : start state
      gamma (float) : discount factor

    Returns:
      v (float) : Returns next state value: Expected return for next iteration, 
                  starting from state s, and then following policy π
    """

    if mdp.is_terminal(state): return 0
    
    v = 0
    actions = mdp.get_possible_actions(state)
    
    for action in actions:
        action_value = get_action_value(mdp, state_values, state, action, gamma)
        if action_value > v:
            v = action_value

    return v

def value_iteration(mdp, gamma, num_iter, min_difference, state_values=None):
    """
    Performs num_iter value iteration steps starting from state_values.

    Args:
      mdp : the MDP that we're using
      gamma (float) : discount factor for MDP
      num_iter (string) : maximum number of iterations
      min_difference (float) : stop Value Iteration if new values are at least this close to old values
      state_values (dict- {string->float}) : state-values for the states

    Returns:
      state_values (dict) : Returns the state-values at the end of Value Iteration
    """

    # Initialize V(s) to 0 if argument is not provided
    state_values = state_values or {s : 0 for s in mdp.get_all_states()}

    new_state_values = {}
    
    for i in range(num_iter):
        for s in state_values:
            new_state_values[s] = get_new_state_value(mdp, state_values, s, gamma)

        # Compute difference
        diff = max(abs(new_state_values[s] - state_values[s]) for s in mdp.get_all_states())
        
        state_values = dict(new_state_values)

        if i % (0.1 * num_iter) == 0:
            print("iter %4i   |   diff: %6.5f   |   " % (i, diff), end="")
            print("  ".join("V(%s) = %.3f" % (s, v) for s,v in state_values.items()), end='\n\n')
        
        # Stopping criteria
        if diff < min_difference:
            print("Terminated")
            break
            
    return state_values

def get_optimal_action(mdp, state_values, state, gamma=1, debug=False):
    """
    Args:
      mdp (MDP): the MDP that we're using
      state_values (dict- {string->float}) : state-values for the states in the current iteration
      state (string) : start state
      gamma (float) : discount factor

    Returns:
      optimal_action (string) : Returns the optimal action - action that results 
                                in the maximum action-value.
    """

    if mdp.is_terminal(state): 
        return None
    
    actions = mdp.get_possible_actions(state)
    
    # Compute optimal action as per formula above. 
    optimal_action = None
    optimal_action_value = - float("inf")
    
    if debug:
        print()
    
    for action in actions:
        action_value = get_action_value(mdp, state_values, state, action, gamma)

        if action_value >= optimal_action_value:
            optimal_action_value = action_value
            optimal_action = action
            
            if debug:
                print('action', action, ':', action_value)

    return optimal_action

In [18]:
transitions_dict = {}
rewards_dict = {}

for curr_s in range(transitions.shape[0]):
    transitions_dict[curr_s] = {}
    rewards_dict[curr_s] = {}
    for a in range(transitions.shape[1]):        
        if math.isclose(sum(transitions[curr_s][a][:]), 0):
            continue
        
        transitions_dict[curr_s][a] = {}
        rewards_dict[curr_s][a] = {}
        for next_s in range(transitions.shape[2]):
            transitions_dict[curr_s][a][next_s] = transitions[curr_s][a][next_s].item()
            rewards_dict[curr_s][a][next_s] = rewards[curr_s][a][next_s].item()

In [19]:
print('Transitions:')
pprint.pprint(transitions_dict)

print('\nRewards:')
pprint.pprint(rewards_dict)

Transitions:
{0: {0: {0: 0.9715876701889555,
         1: 0.0004530371490462218,
         2: 9.000738060520963e-05,
         3: 0.0,
         4: 1.5001230100868271e-05,
         5: 0.0022621854992109355,
         6: 6.900565846399404e-05,
         7: 0.019075564196264095,
         8: 0.006447528697353183},
     1: {0: 0.2490842490842491,
         1: 0.0,
         2: 0.0,
         3: 0.0,
         4: 0.0,
         5: 0.4358974358974359,
         6: 0.10622710622710622,
         7: 0.08058608058608059,
         8: 0.1282051282051282},
     2: {0: 0.4060073559460564,
         1: 0.0016346546791990192,
         2: 0.0012259910093992644,
         3: 0.0,
         4: 0.0,
         5: 0.3201879852881079,
         6: 0.1103391908459338,
         7: 0.07887208827135268,
         8: 0.08173273395995095},
     3: {0: 0.37142857142857144,
         1: 0.0,
         2: 0.0380952380952381,
         3: 0.0,
         4: 0.0,
         5: 0.2,
         6: 0.009523809523809525,
         7: 0.15238095238095

         5: 0.05470755099284074,
         6: 0.07778378134990319,
         7: 0.17339817191228782,
         8: 0.6194785897609077},
     1: {0: 0.0,
         1: 0.0,
         2: 0.0,
         3: 0.0,
         4: 0.0,
         5: 0.09216589861751152,
         6: 0.2341589861751152,
         7: 0.008352534562211982,
         8: 0.6653225806451613},
     2: {0: 0.00030321406913280777,
         1: 0.00010107135637760258,
         2: 0.0,
         3: 0.0,
         4: 0.0,
         5: 0.17586416009702852,
         6: 0.16838487972508592,
         7: 0.029613907418637557,
         8: 0.6257327673337376},
     3: {0: 0.0,
         1: 0.0,
         2: 0.0,
         3: 0.0,
         4: 0.0,
         5: 0.25,
         6: 0.0,
         7: 0.0,
         8: 0.75},
     4: {0: 0.00014443909484833896,
         1: 0.0,
         2: 0.0,
         3: 0.0,
         4: 0.0,
         5: 0.16492537313432837,
         6: 0.3262156957149735,
         7: 0.012132883967260472,
         8: 0.4965816080885893},
   

         7: -0.019654954385331335,
         8: 0.023365549634179084}},
 7: {0: {0: 0.011492306828391809,
         1: 0.023290094339622536,
         2: 0.0,
         3: 0.0,
         4: 0.0,
         5: 0.016436433063791656,
         6: 0.026901954177897668,
         7: 0.322266116352134,
         8: 0.038986410601977425},
     1: {0: 0.0,
         1: 0.0,
         2: 0.0,
         3: 0.0,
         4: 0.0,
         5: 0.3631284916201115,
         6: 1.896983240223464,
         7: 0.5333120510774145,
         8: 0.3729608938547484},
     2: {0: -0.001662377275760175,
         1: -0.0007129920884567724,
         2: 0.0,
         3: 0.0,
         4: 0.0,
         5: 0.18778762749022704,
         6: 0.04518730340291593,
         7: 0.08772852921552121,
         8: 0.19309884663044116},
     3: {0: 0.0,
         1: 0.0,
         2: 0.0,
         3: 0.0,
         4: 0.0,
         5: 0.5076923076923077,
         6: 0.0,
         7: 1.5046153846153847,
         8: 0.716923076923077},
     4: {0

# Run KDD Marketing RL

In [20]:
import random

initial_states = np.arange(num_states)
mdp = MDP(transitions_dict, rewards_dict, initial_state=random.choice(initial_states), seed=1)

In [21]:
mdp.reset()

# Discount factor γ / gamma
# If γ=0, the agent will be completely myopic and only learn about actions that produce an immediate reward. 
# If γ=1, the agent will evaluate each of its actions based on the sum total of all of its future rewards.

gamma = 0.95
num_iter = 1000
min_difference = 1e-5

state_values = value_iteration(mdp, gamma, num_iter, min_difference)

iter    0   |   diff: 3.23413   |   V(0) = 1.418  V(1) = 0.790  V(2) = 0.661  V(3) = 3.234  V(4) = 0.499  V(5) = 3.142  V(6) = 3.127  V(7) = 1.271  V(8) = 1.875

iter  100   |   diff: 0.01638   |   V(0) = 52.609  V(1) = 52.103  V(2) = 47.884  V(3) = 53.056  V(4) = 42.578  V(5) = 56.113  V(6) = 55.531  V(7) = 53.299  V(8) = 53.128

iter  200   |   diff: 0.00010   |   V(0) = 52.918  V(1) = 52.412  V(2) = 48.193  V(3) = 53.365  V(4) = 42.887  V(5) = 56.422  V(6) = 55.840  V(7) = 53.608  V(8) = 53.437

Terminated


## Results

In [22]:
print("Final MDP state values after running Value Iteration:")
pprint.pprint(state_values)

Final MDP state values after running Value Iteration:
{0: 52.91958357565906,
 1: 52.41384045057953,
 2: 48.19487661419275,
 3: 53.36688150713131,
 4: 42.88875034049442,
 5: 56.42366862890062,
 6: 55.841928487719066,
 7: 53.609764776116705,
 8: 53.43862327311916}


In [23]:
print("Optimal action for each MDP state")

for state in range(num_states):
    value = state_values[state]
    optimal_action_id = get_optimal_action(mdp, state_values, state, gamma, debug=True)
    print("    MDP State", state, ": optimal action ", optimal_action_id, ACTION_NAMES[optimal_action_id])

Optimal action for each MDP state

action 0 : 50.32561042600739
action 1 : 52.91959273662287
    MDP State 0 : optimal action  1 labels_only

action 0 : 49.82935026832865
action 1 : 52.21336558795234
action 2 : 52.413849611543334
    MDP State 1 : optimal action  2 thank_you_with_labels

action 0 : 46.042536367503914
action 1 : 48.19488577515655
    MDP State 2 : optimal action  1 labels_only

action 0 : 50.45030232945953
action 1 : 50.692970945080596
action 3 : 53.36689066809511
    MDP State 3 : optimal action  3 calendars_with_stickers

action 0 : 42.88875950145822
    MDP State 4 : optimal action  0 no_action

action 0 : 52.35581749327586
action 1 : 56.42367778986442
    MDP State 5 : optimal action  1 labels_only

action 0 : 51.65717612312266
action 1 : 55.84193764868287
    MDP State 6 : optimal action  1 labels_only

action 0 : 50.82420590388572
action 1 : 53.609773937080504
    MDP State 7 : optimal action  1 labels_only

action 0 : 51.36646737025541
action 1 : 53.4386324340829

In [24]:
pickle.dump(transitions_dict, open(data_dir + 'mdp_transitions_dict_nS=' + str(num_states) + '_nA=' + str(num_actions) + '.p', "wb"))
pickle.dump(rewards_dict, open(data_dir + 'mdp_rewards_dict_nS=' + str(num_states) + '_nA=' + str(num_actions) + '.p', "wb"))
pickle.dump(state_values, open(data_dir + 'mdp_state_values_nS=' + str(num_states) + '_nA=' + str(num_actions) + '.p', "wb"))

In [25]:
# num_states = 6
# num_actions = 4

# transitions_dict = pickle.load(open('./mdp_data/transitions_dict' + str(num_states) + '.p', "rb"))
# rewards_dict = pickle.load(open('./mdp_data/rewards_dict' + str(num_states) + '.p', "rb"))
# state_values = pickle.load(open('./mdp_data/state_values' + str(num_states) + '.p', "rb"))

## Evaluate policy

In [26]:
EPISODE_LENGTH = 25 # total num months / TIME_PERIOD of each event

def get_average_reward(start_state, mdp, state_values, gamma):
    """
    Returns the average reward for the episode of length EPISODE_LENGTH 
    starting at `start_state`, when following the OPTIMAL policy. 
    """
    s = start_state

    rewards = []
    for i in range(EPISODE_LENGTH):
        s, r, done, _ = mdp.step(get_optimal_action(mdp, state_values, s, gamma))
        rewards.append(r)
    
    return np.mean(rewards)

In [27]:
# Measure agent's average reward over random episodes
num_samples = 10000

avg_rewards = []
for _ in range(num_samples):
    start_s = mdp.reset() # select random start state

    r = get_average_reward(start_s, mdp, state_values, gamma)
    avg_rewards.append(r)

print("Average reward over", num_samples, "episodes:", np.mean(avg_rewards))

Average reward over 10000 episodes: 2.4465294480158457


## Compare policies

In [28]:
mdp_episodes = mdp_data[['id', 'action', 'reward', 'bloc', 'state']]

sample_episodes = mdp_episodes
display(sample_episodes)

,id,action,reward,bloc,state
0,95515,5,0.00,0,5
1,95515,6,-0.68,1,0
2,95515,0,0.00,2,0
3,95515,6,-0.68,3,0
4,95515,2,-0.68,4,0
...,...,...,...,...,...
2385295,185114,0,0.00,20,5
2385296,185114,0,0.00,21,0
2385297,185114,0,0.00,22,0
2385298,185114,0,0.00,23,0


In [29]:
# Measure agent's average reward following original data policy

print("Average reward for original dataset policy: ", np.mean(sample_episodes['reward']))

Average reward for original dataset policy:  0.5396969018572084


In [30]:
# Measure agent's average reward following RL optimal policy
T = 50 # length of episode

avg_rewards = []
for i, row in sample_episodes.iterrows():
    if row['bloc'] != 0:
        continue
    
    if i % 200000 == 0:
        print('========================== Processed', i, 'rows (', i / len(sample_episodes) * 100,'%)==========================', (time.time() - start_time) / 60, 'mins')
  
    s = row['state']
    mdp.reset(s) # Set start state to `s`

    r = get_average_reward(s, mdp, state_values, gamma)
    avg_rewards.append(r)

print("Average reward for optimal RL policy: ", np.mean(avg_rewards))

========================== Processed 0 rows ( 0.0 %)========================== 45.77112923463186 mins
========================== Processed 200000 rows ( 8.384689556869157 %)========================== 46.886274337768555 mins
========================== Processed 400000 rows ( 16.769379113738314 %)========================== 47.91885209083557 mins
========================== Processed 600000 rows ( 25.154068670607472 %)========================== 48.93583238919576 mins
========================== Processed 800000 rows ( 33.53875822747663 %)========================== 50.58500609397888 mins
========================== Processed 1000000 rows ( 41.92344778434578 %)========================== 51.62919533650081 mins
========================== Processed 1200000 rows ( 50.308137341214945 %)========================== 52.63206286430359 mins
========================== Processed 1400000 rows ( 58.69282689808409 %)========================== 53.63718146085739 mins
========================== Processed 1600000

In [31]:
# Measure agent's average reward following random policy

avg_rewards = []
for i, row in sample_episodes.iterrows():
    if row['bloc'] != 0:
        continue
    
    if i % 200000 == 0:
        print('========================== Processed', i, 'rows (', i / len(sample_episodes) * 100,'%)==========================', (time.time() - start_time) / 60, 'mins')
  
    
    s = row['state']
    mdp.reset(state=s) # Set start state to `s`

    rewards = []
    for i in range(T):
        random_action = random.choice(mdp.get_possible_actions(s))
        
        s, r, done, _ = mdp.step(random_action)
        rewards.append(r)
    
    avg_rewards.append(np.mean(rewards))

print("Average reward for random policy: ", np.mean(avg_rewards))

========================== Processed 0 rows ( 0.0 %)========================== 58.424634365240735 mins
========================== Processed 200000 rows ( 8.384689556869157 %)========================== 59.24589880307516 mins
========================== Processed 400000 rows ( 16.769379113738314 %)========================== 60.10864270925522 mins
========================== Processed 600000 rows ( 25.154068670607472 %)========================== 60.961403874556225 mins
========================== Processed 800000 rows ( 33.53875822747663 %)========================== 61.761261089642844 mins
========================== Processed 1000000 rows ( 41.92344778434578 %)========================== 62.54646452267965 mins
========================== Processed 1200000 rows ( 50.308137341214945 %)========================== 63.46577716271083 mins
========================== Processed 1400000 rows ( 58.69282689808409 %)========================== 64.35233932733536 mins
========================== Processed 16000

## Save data

In [32]:
mdp_data.to_csv(data_dir + 'mdp_data_nS=' + str(num_states) + '_nA=' + str(num_actions) + '.csv', index=False)
mdp_episodes.to_csv(data_dir + 'mdp_episodes_nS=' + str(num_states) + '_nA=' + str(num_actions) + '.csv', index=False)

In [33]:
# num_states = 6
# num_actions = 4

# mdp_data = pd.read_csv('./rl_data/mdp_data_nS=' + str(num_states) + '_nA=' + str(num_actions) + '.csv', index=False)
# mdp_episodes = pd.read_csv('./rl_data/mdp_episodes_nS=' + str(num_states) + '_nA=' + str(num_actions) + '.csv', index=False)